In [7]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [1]:
#initiate spark instance 
import pyspark
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
#Create SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("SparkApp") \
    .getOrCreate()

#spark.stop()
print("spark success")

24/06/06 19:44:51 WARN Utils: Your hostname, RUIIs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.5 instead (on interface en0)
24/06/06 19:44:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/06 19:44:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


spark success


In [16]:
#print current directory
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print(f"Current Working Directory: {current_directory}")


Current Working Directory: /Users/ruiiyoung/Downloads/MacAir2016_Desktop/IT stuff DIY +所有自己上的课/U St Andrews Data Science PGDip/5939data-driven system/assignment1/SparkA1


24/06/07 07:15:00 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

In [6]:
#part 1:1.load all csv in this same path
df= spark.read.format("csv").option("header", True).option("inferSchema", True).load("/Users/ruiiyoung/Downloads/MacAir2016_Desktop/IT stuff DIY +所有自己上的课/U St Andrews Data Science PGDip/5939data-driven system/assignment1/lab_dataset.csv")
df.show()
display(df)

+---------------+-----------+-------+-------------------+----------+---------------+--------------------+--------------------+-----------------+--------------------+--------------------+-------------+-----------------+---------------+--------+------------------+--------------------+--------------------+---------------+--------------------+--------------------+---------+----------+---------------+----------------+-----------+------------+------+---------------------+--------------------------------+-------------------------------------+---------------------------------+--------------------------------------+-----------------------------+-----------------+---------+----------------+-----------------+--------+
| IncidentNumber| DateOfCall|CalYear|         TimeOfCall|HourOfCall|  IncidentGroup| StopCodeDescription|  SpecialServiceType| PropertyCategory|        PropertyType|    AddressQualifier|Postcode_full|Postcode_district|           UPRN|    USRN|IncGeo_BoroughCode|  IncGeo_BoroughName|

DataFrame[IncidentNumber: string, DateOfCall: string, CalYear: int, TimeOfCall: timestamp, HourOfCall: int, IncidentGroup: string, StopCodeDescription: string, SpecialServiceType: string, PropertyCategory: string, PropertyType: string, AddressQualifier: string, Postcode_full: string, Postcode_district: string, UPRN: double, USRN: int, IncGeo_BoroughCode: string, IncGeo_BoroughName: string, ProperCase: string, IncGeo_WardCode: string, IncGeo_WardName: string, IncGeo_WardNameNew: string, Easting_m: int, Northing_m: int, Easting_rounded: int, Northing_rounded: int, Latitude: double, Longitude: double, FRS: string, IncidentStationGround: string, FirstPumpArriving_AttendanceTime: int, FirstPumpArriving_DeployedFromStation: string, SecondPumpArriving_AttendanceTime: int, SecondPumpArriving_DeployedFromStation: string, NumStationsWithPumpsAttending: int, NumPumpsAttending: int, PumpCount: int, PumpHoursRoundUp: int, Notional Cost (£): int, NumCalls: int]

In [11]:
#part1:2 Allow the user to search the dataset by the borough name, showing the total number of callouts in 2020.
#搜索功能还没有写
# Filter the DataFrame to include only data for the year 2020
df_2020 = df.filter(col('CalYear') == 2020)
# Group the data by 'IncGeo_BoroughName' and sum the 'NumbCalls' column
df_Borough_call = df_2020.groupBy('IncGeo_BoroughName').sum('NumCalls')
df_Borough_call.show(truncate=False)

+----------------------+-------------+
|IncGeo_BoroughName    |sum(NumCalls)|
+----------------------+-------------+
|HAMMERSMITH AND FULHAM|3285         |
|CROYDON               |5638         |
|GREENWICH             |3893         |
|TOWER HAMLETS         |5513         |
|CAMDEN                |5009         |
|ENFIELD               |4387         |
|HOUNSLOW              |3895         |
|BEXLEY                |3240         |
|LEWISHAM              |4755         |
|REDBRIDGE             |3424         |
|CITY OF LONDON        |1040         |
|LAMBETH               |5224         |
|HARROW                |2176         |
|WESTMINSTER           |8283         |
|ISLINGTON             |3934         |
|RICHMOND UPON THAMES  |2128         |
|BROMLEY               |3806         |
|SOUTHWARK             |5595         |
|HARINGEY              |4243         |
|WALTHAM FOREST        |3725         |
+----------------------+-------------+
only showing top 20 rows



In [ ]:
#part1: 3  Given a range of hours (for example between 1-2 hours), show the user all incidents where the LFB spent this much time dealing with it.
HourOfCall = 13, 

In [15]:
#part1: 4 Calculate the number of incidents in 2021 where there were at least 2 fire pumps deployed.
# Filter DataFrame where PumpCount > 2
filtered_df = df.filter((col("PumpCount") >2 ) & (col("CalYear") == 2021)) # >= 2 应该要用一个or

# Count distinct PumpCount values
IncidentNumber = filtered_df.select("IncidentNumber").distinct().count()
print(f"Number of incidents is  {IncidentNumber} in 2021 where there were at least 2 fire pumps deployed ")


Number of incidents is  9077 in 2021 where there were at least 2 fire pumps deployed 


24/06/07 07:14:20 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

In [9]:
#part1:5.Which type of property had the most incident callouts during 2022?


from pyspark.sql.functions import col
from pyspark.sql import functions as F
# Filter the DataFrame to include only data for the year 2020
df_2022 = df.filter(col('CalYear') == 2022)
# property 
df_Property = df_2020.groupBy('PropertyType').agg(F.sum('NumCalls').alias('TotalNumCalls'))
df_Property = df_Property.orderBy(F.col('TotalNumCalls').desc())
df_Property.show(truncate=False) #这里printresult 真的要用print


+-----------------------------------------------------+-------------+
|PropertyType                                         |TotalNumCalls|
+-----------------------------------------------------+-------------+
|Purpose Built Flats/Maisonettes - 4 to 9 storeys     |17451        |
|House - single occupancy                             |14676        |
|Purpose Built Flats/Maisonettes - Up to 3 storeys    |11608        |
|Car                                                  |7377         |
|Self contained Sheltered Housing                     |6097         |
|Purpose Built Flats/Maisonettes - 10 or more storeys |4977         |
|Converted Flat/Maisonettes - 3 or more storeys       |4501         |
|Converted Flat/Maisonette - Up to 2 storeys          |3379         |
|Purpose built office                                 |3221         |
|Domestic garden (vegetation not equipment)           |2665         |
|Single shop                                          |2635         |
|Small refuse/rubbis

24/04/28 06:49:38 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 910470 ms exceeds timeout 120000 ms
24/04/28 06:49:38 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/28 06:49:45 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
# Using an appropriate Python library, 
# draw a graph showing the trend of incident callouts on the road between 2020 and 2022. 
# Make sure to define how you interpret a road incident based on the dataset.